In [98]:
import numpy as np
import pandas as pd
import json


from sqlalchemy import create_engine, select, literal_column, text
from sqlalchemy import Column,  String,  ARRAY

from sqlalchemy.sql import func, distinct
from sqlalchemy.dialects.postgresql import array
from sqla_schema import *

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import os

In [8]:
def query_to_df(Q):
    df = pd.read_sql(Q.statement, Q.session.bind)
    return df

In [86]:
# Get the sessions for which we have spike data
Q = session.\
    query(Session).\
    join(SessionProbe,SessionProbe.session_id == Session.id).\
    join(Channel,Channel.session_probe_id == SessionProbe.id).\
    join(Unit,Unit.channel_id == Channel.id).\
    filter(UnitSpikeTimes.unit_id==Unit.id).group_by(Session.id)
query_df = query_to_df(Q)

In [93]:
for session_id  in query_df.id:


721123822
719161530
715093703


In [99]:
data_directory = 'C:\\Users\\yoni.browning\\Documents\\DataJoint\\AllenData'
manifest_path = os.path.join(data_directory, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)


In [102]:
session = cache.get_session_data(session_id)        
rf_stim_table = session.stimulus_presentations
rf_stim_table.reset_index(inplace=True)

In [107]:
rf_stim_table.insert(0,'session_id',(session.ecephys_session_id*np.ones(len(rf_stim_table))).astype(int))